We import scipy package for cosine similarity computation, read the anime and cleaned ratings data files.

In [ ]:
import math
import sys
import csv
from scipy import spatial
#import numpy as np

with open('anime.csv', 'r') as f:
    reader = csv.reader(f)
    anime_list = list(reader)
del anime_list[0]

with open('cleanedratings.csv', 'r') as f:
    reader = csv.reader(f)
    rating_list = list(reader)
#del rating_list[0]
#print(rating_list[:20])

Initialize ratingsmatrix with -2 (not watched and not rated)

In [ ]:
ratingsmatrix = [[-2]*len(anime_list) for _ in range(1000)]
#print(len(ratingsmatrix[100]))
#print(ratingsmatrix[0])
#print(ratingsmatrix[0][0])

Fill the Ratings Matrix with the ratings list. The users represent the rows in the Ratings Matrix and the movies are the columns. (userid-1) represents the corresponding user row. 

In [ ]:
#print(ratingsmatrix[0][0])
count=0
for x in range (0,len(rating_list)):
    row=int(rating_list[x][0])-1
    column=int(rating_list[x][1])
    ratingsmatrix[row][column]=int(rating_list[x][2])
    #print(row)
    #print(column)
#print(count)
#print((ratingsmatrix[0]))

We now find the average of the rated animes and store it in average list

In [ ]:
average=[]
for x in range (0,len(ratingsmatrix)):
    count=0
    avg=0
    for y in ratingsmatrix[x]:
        if y!=-1 and y!=-2:
            avg=avg+y
            count=count+1
    if count==0:
        average.append(0)
    else:
        average.append(avg/count)

Now we normalize the user ratings around 0 because some users are hard raters and some are soft raters. To do that we take the average of all ratings and substract the average from each rating.

In [ ]:
for x in range (0,len(ratingsmatrix)):
    for y in range (0,len(ratingsmatrix[x])):
        if ratingsmatrix[x][y]!=-2 and ratingsmatrix[x][y]!=-1:
            ratingsmatrix[x][y]=ratingsmatrix[x][y]-average[x]

Anime which has been watched but not rated(the value -1) is given a value slightly above the average user rating (0 after normalization). We did this because from our data analysis we can conclude that most users only rate anime they like. They usually dont rate average anime. We are also doing this because it will cause the unrated watched anime to atleast have some effect on the recommendation, since it is the users preference. If we don't do this, there would be no difference between watched unrated anime and unwatched anime.

Hence after normalization, unwatched anime (-2) is treated as average rated anime (zero since we have normalized around 0) and watched unrated anime is given a value slightly above the average(0.1)

In [ ]:
for x in range(0,len(ratingsmatrix)):
    for y in range (0,len(ratingsmatrix[x])):
        #print(rrow,rcol)
        if ratingsmatrix[x][y]==-2:
            ratingsmatrix[x][y]=0
        elif ratingsmatrix[x][y]==-1:
            ratingsmatrix[x][y]=0.1
        else:
            ratingsmatrix[x][y]= ratingsmatrix[x][y]+0.1
#print(x)

Similarity matrix is a User x User matrix with the corresponding similarities as the cell values. We initialize it with -2.
We then calculate the cosine similarities between the user vectors and fill the cell values.

In [ ]:
similaritymatrix = [[-2]*1000 for _ in range(1000)]

for x in range(0,len(ratingsmatrix)):
    for y in range(0,len(ratingsmatrix)):
        if similaritymatrix[x][y]==-2:
            if x==y:
                similaritymatrix[x][y]=1
            else:
                result=1 - spatial.distance.cosine(ratingsmatrix[x], ratingsmatrix[y])
                similaritymatrix[x][y]=result
                similaritymatrix[y][x]=result
    #print(x)
#print(result)

Output the similarity matrix as a file. We precompute and store it so that recommendations can be done in real time

In [ ]:
with open("similarity.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(similaritymatrix)